# AutoHDR Lens Correction — Sprint Notebook

**36-hour hackathon** | Displacement field model for automatic lens distortion correction

**Scoring:** Edge (40%) > Line (22%) > Gradient (18%) > SSIM (15%) > Pixel (5%)

**Deadline:** Sunday, Feb 22, 2026, 2:00 PM

| Gate | Deadline | Rule |
|------|----------|------|
| First Kaggle submission | **Hour 10** | NON-NEGOTIABLE |
| Architecture lock | **Hour 22** | Refinement only |
| Experiment lock | **Hour 30** | Ship only |

In [ ]:
# ============================================================
# 1. GPU Check & Environment
# ============================================================
!nvidia-smi
import torch
print(f"\nPyTorch: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_mem / 1e9:.1f} GB")

In [ ]:
# ============================================================
# 2. Mount Google Drive
# ============================================================
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# ============================================================
# 3. Clone Repo & Install Dependencies
# ============================================================
import os

WORK_DIR = '/content/drive/MyDrive/autohdr'
REPO_URL = 'https://github.com/alediez2048/AutoHDRAgents.git'

if not os.path.exists(WORK_DIR):
    !git clone {REPO_URL} {WORK_DIR}
else:
    !cd {WORK_DIR} && git pull

os.chdir(WORK_DIR)
!pip install -q -r requirements.txt

print(f"\nWorking directory: {os.getcwd()}")
print("\nFiles:")
!ls -la *.py *.sh *.txt

In [ ]:
# ============================================================
# 4. Kaggle Setup & Data Download
# ============================================================
import os

# OPTION A: Set Kaggle credentials (edit these)
os.environ['alediez2408'] = 'alediez2408'  # <-- EDIT THIS
os.environ['KGAT_e7d8c75d20533b7a5affffe513cfa6da'] = 'KGAT_e7d8c75d20533b7a5affffe513cfa6da'            # <-- EDIT THIS

!pip install -q kaggle

# Uncomment and edit the competition name:
# !kaggle competitions download -c COMPETITION_NAME -p data/
# !unzip -q -o data/*.zip -d data/

# OPTION B: If data is already on Drive, symlink it:
# !ln -sf /content/drive/MyDrive/YOUR_DATA_FOLDER data

print("\nData directory contents:")
!ls -la data/ 2>/dev/null || echo "No data yet -- download or link your data above"

In [ ]:
# ============================================================
# 5. Inspect Training Data
# ============================================================
import sys
sys.path.insert(0, '.')

from data import discover_pairs
from utils import set_seed
import cv2
import numpy as np
import matplotlib.pyplot as plt
import random

set_seed(42)
pairs = discover_pairs('data')
print(f"Found {len(pairs)} image pairs")

if len(pairs) > 0:
    # Show 6 random pairs
    samples = random.sample(pairs, min(6, len(pairs)))
    fig, axes = plt.subplots(len(samples), 2, figsize=(12, 4 * len(samples)))
    if len(samples) == 1:
        axes = [axes]
    for i, (dist_path, corr_path) in enumerate(samples):
        dist = cv2.cvtColor(cv2.imread(dist_path), cv2.COLOR_BGR2RGB)
        corr = cv2.cvtColor(cv2.imread(corr_path), cv2.COLOR_BGR2RGB)
        axes[i][0].imshow(dist)
        axes[i][0].set_title(f'Distorted {dist.shape}')
        axes[i][1].imshow(corr)
        axes[i][1].set_title(f'Corrected {corr.shape}')
        axes[i][0].axis('off')
        axes[i][1].axis('off')
    plt.tight_layout()
    os.makedirs('outputs', exist_ok=True)
    plt.savefig('outputs/data_inspection.png', dpi=100)
    plt.show()

    # Resolution stats
    heights, widths = [], []
    for d, c in pairs[:100]:
        img = cv2.imread(d)
        if img is not None:
            heights.append(img.shape[0])
            widths.append(img.shape[1])
    print(f"\nResolution stats (first {len(heights)} images):")
    print(f"  Height: min={min(heights)}, max={max(heights)}, mean={np.mean(heights):.0f}")
    print(f"  Width:  min={min(widths)}, max={max(widths)}, mean={np.mean(widths):.0f}")
else:
    print("No pairs found. Check your data directory structure.")
    print("Expected: data/distorted/ and data/corrected/ with matching filenames")

In [ ]:
# ============================================================
# 6. Run Sanity Suite (All 7 Checks)
# ============================================================
# ALL checks must PASS before training begins
!python validate.py

In [ ]:
# ============================================================
# 7. S01 Baseline Training (512x512)
# ============================================================
# Coarse geometry learning: 30-50 epochs at 512x512
# Expected: ~3-4 hours on L4
# HARD STOP AT HOUR 9 -- move to inference regardless
!python train.py --stage s01

In [ ]:
# ============================================================
# 8. S01 Inference & First Submission (HOUR 10 GATE)
# ============================================================
# NON-NEGOTIABLE: Must have a valid Kaggle submission by Hour 10
import glob
import os

ckpt_dir = '/content/drive/MyDrive/autohdr_checkpoints'
best_ckpt = os.path.join(ckpt_dir, 'best.pth')

if not os.path.exists(best_ckpt):
    ckpts = sorted(glob.glob(os.path.join(ckpt_dir, '*.pth')))
    best_ckpt = ckpts[-1] if ckpts else None
    print(f"No best.pth found, using latest: {best_ckpt}")
else:
    print(f"Using best checkpoint: {best_ckpt}")

!python inference.py --checkpoint "{best_ckpt}" --test_dir data/test --output_dir outputs/test_s01 --zip

print("\n" + "=" * 60)
print("SUBMISSION STEPS:")
print("1. Upload outputs/test_s01.zip to https://bounty.autohdr.com")
print("2. Download submission.csv from scoring service")
print("3. Submit CSV to Kaggle leaderboard")
print("4. Record public score in the Scoreboard Log below")
print("=" * 60)

In [ ]:
# ============================================================
# 9. S02 Progressive Resolution (768x768)
# ============================================================
# Fine-tune best S01 at 768x768: 10-15 epochs, lr=5e-5
# Expected: ~2-3 hours
# WARNING: Do NOT attempt 1024 on L4 -- OOM likely
import os
best_ckpt = os.path.join('/content/drive/MyDrive/autohdr_checkpoints', 'best.pth')
!python train.py --stage s02 --resume "{best_ckpt}"

In [ ]:
# ============================================================
# 10. S02 Inference & Submit
# ============================================================
import os
best_ckpt = os.path.join('/content/drive/MyDrive/autohdr_checkpoints', 'best.pth')
print(f"Using checkpoint: {best_ckpt}")

!python inference.py --checkpoint "{best_ckpt}" --test_dir data/test --output_dir outputs/test_s02 --zip

print("\nUpload outputs/test_s02.zip to bounty.autohdr.com -> CSV -> Kaggle")

## Scoreboard Log

| Run | Config Delta | Local Edge | Local SSIM | Local MAE | Public MAE | Timestamp |
|-----|-------------|-----------|-----------|----------|-----------|----------|
| S01 | baseline 512 | | | | | Hour __ |
| S02 | prog 768 | | | | | Hour __ |
| X01 | edge wt 0.50 | | | | | Hour __ |
| X02 | grad wt 0.25 | | | | | Hour __ |

In [ ]:
# ============================================================
# 11. Git Checkpoint
# ============================================================
# Run after each major milestone to save progress
!git add -A && git commit -m "[TRAIN] checkpoint: S01+S02 training complete" && git push

## Decision Rules Reminder

1. **No new architecture after Hour 22** -- lock what works, refinement only
2. **No unlogged experiments** -- every run gets config + metric entry
3. **Tail risk beats average gains** -- reject changes that worsen worst-decile
4. **Hard-fail veto** -- any run with increased 0.0-score risk is rejected
5. **First submission by Hour 10** -- non-negotiable
6. **Identity fallback always available** -- input unchanged > hard-fail 0.0
7. **Last 6 hours are sacred** -- no experiments, just ship

### Submission Pipeline
```
inference.py -> outputs/*.png -> ZIP -> bounty.autohdr.com -> submission.csv -> Kaggle
```

In [ ]:
# ============================================================
# 12. X-Track Experiments (Hour 12-22 only)
# ============================================================
# Uncomment ONE experiment at a time. Single variable changes only.

# X01: Edge loss weight 0.40 -> 0.50
# !python train.py --stage s01 --resume "{best_ckpt}" --config loss_weights.edge=0.50 loss_weights.l1_pixel=0.05

# X02: Gradient loss weight 0.20 -> 0.25
# !python train.py --stage s01 --resume "{best_ckpt}" --config loss_weights.grad_orientation=0.25 loss_weights.l1_pixel=0.05

In [ ]:
# ============================================================
# 13. Final Submission (Hour 30-36)
# ============================================================
# NO EXPERIMENTS. NO TRAINING. SHIP ONLY.
import os
best_ckpt = os.path.join('/content/drive/MyDrive/autohdr_checkpoints', 'best.pth')

# Final inference with best model
!python inference.py --checkpoint "{best_ckpt}" --test_dir data/test --output_dir outputs/test_final --zip

# Reproducibility check -- run again, outputs must be identical
!python inference.py --checkpoint "{best_ckpt}" --test_dir data/test --output_dir outputs/test_verify --zip

# Compare
import filecmp
import glob
final_files = sorted(glob.glob('outputs/test_final/*.png'))
verify_files = sorted(glob.glob('outputs/test_verify/*.png'))
match = all(filecmp.cmp(f, v) for f, v in zip(final_files, verify_files))
print(f"\nReproducibility check: {'PASS -- outputs identical' if match else 'FAIL -- outputs differ!'}")

print("\nFINAL SUBMISSION STEPS:")
print("1. Upload outputs/test_final.zip to bounty.autohdr.com")
print("2. Submit CSV to Kaggle")
print("3. Record video (< 1 min)")
print("4. Submit video + code via Google Form")
print("5. Final git push")
!git add -A && git commit -m "[SHIP] final: best model submitted" && git push